In [1]:
from src.classifier import DNNClassifier
from src.encoders import BertEncoder, TFIDFEncoder, DebertaEncoder, RobertaEncoder
from src.data import NewsDataset
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import torch

In [2]:
train_data = pd.read_csv('data/train_augmented_v2.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_data["label"].value_counts()

label
Politics         103
Health           103
Finance          103
Travel           103
Food             103
Education        103
Environment      103
Fashion          103
Science          103
Sports           103
Technology       103
Entertainment    103
Name: count, dtype: int64

In [4]:
encoder = RobertaEncoder()
label_encoder = LabelEncoder()
X_train = torch.Tensor(encoder.fit_transform(train_data["text"].to_list()))
y_train = torch.Tensor(label_encoder.fit_transform(train_data["label"])).long()
X_test = torch.Tensor(encoder.transform(test_data["text"].to_list()))
y_test = torch.Tensor(label_encoder.transform(test_data["label"])).long()

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1140/1140 [01:26<00:00, 13.21it/s]


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = NewsDataset(X_train, y_train, device)
test_dataset = NewsDataset(X_test, y_test, device)

In [6]:
# Create DataLoader
from torch.utils.data import DataLoader
# Get train and test data in form of Dataloader class
train_loader = DataLoader(train_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [7]:
classifier = DNNClassifier(n_classes=12,input_size=X_train.shape[1]).to(device)
epochs = 300
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters())

In [8]:

from tqdm import tqdm
from datetime import datetime
# Define the dictionary "history" that will collect key performance indicators during training
history = {}
history["epoch"]=[]
history["train_loss"]=[]
history["valid_loss"]=[]
history["train_accuracy"]=[]
history["valid_accuracy"]=[]

# Measure time for training
start_time = datetime.now()
past_val_accuracy = 0
best_model = classifier.state_dict()
# Loop on epochs
for e in range(epochs):
    
    # Set mode in train mode
    classifier.train()
    
    train_loss = 0.0
    train_accuracy = []
    
    # Loop on batches
    for X, y in tqdm(train_loader,desc='Training'):
        
        # Get prediction & loss
        prediction = classifier(X)
        loss = criterion(prediction, y)
        
        # Adjust the parameters of the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        prediction_index = prediction.argmax(axis=1)
        accuracy = (prediction_index==y)
        train_accuracy += accuracy
    
    train_accuracy = (sum(train_accuracy) / len(train_accuracy)).item()
    
    # Calculate the loss on the test data after each epoch
    # Set mode to evaluation (by opposition to training)
    classifier.eval()
    valid_loss = 0.0
    valid_accuracy = []
    for X, y in tqdm(test_loader,desc='Validation'):
         
        prediction = classifier(X)
        
        loss = criterion(prediction, y)

        valid_loss += loss.item()
        
        prediction_index = prediction.argmax(axis=1)
        accuracy = (prediction_index==y)
        valid_accuracy += accuracy
    valid_accuracy = (sum(valid_accuracy) / len(valid_accuracy)).item()
    if valid_accuracy > past_val_accuracy:
        past_val_accuracy = valid_accuracy
        best_model = classifier.state_dict()
    
    # Populate history
    history["epoch"].append(e+1)
    history["train_loss"].append(train_loss / len(train_loader))
    history["valid_loss"].append(valid_loss / len(test_loader))
    history["train_accuracy"].append(train_accuracy)
    history["valid_accuracy"].append(valid_accuracy)    
        
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_loader) :10.3f} \t\t Validation Loss: {valid_loss / len(test_loader) :10.3f}')
    print(f'\t\t Training Accuracy: {train_accuracy :10.3%} \t\t Validation Accuracy: {valid_accuracy :10.3%}')
    
# Measure time for training
end_time = datetime.now()
training_time_pt = (end_time - start_time).total_seconds()

Validation: 100%|██████████| 36/36 [00:00<00:00, 4428.00it/s]


Epoch 1 		 Training Loss:      2.581 		 Validation Loss:      2.484
		 Training Accuracy:     5.097% 		 Validation Accuracy:     8.509%


Validation: 100%|██████████| 36/36 [00:00<00:00, 3146.06it/s]


Epoch 2 		 Training Loss:      2.494 		 Validation Loss:      2.464
		 Training Accuracy:    12.864% 		 Validation Accuracy:    22.368%


Validation: 100%|██████████| 36/36 [00:00<00:00, 4330.60it/s]


...

Epoch 298 		 Training Loss:      0.103 		 Validation Loss:      1.059
		 Training Accuracy:    99.110% 		 Validation Accuracy:    72.018%


Validation: 100%|██████████| 36/36 [00:00<00:00, 4392.84it/s]


Epoch 299 		 Training Loss:      0.110 		 Validation Loss:      1.048
		 Training Accuracy:    98.058% 		 Validation Accuracy:    72.632%


Validation: 100%|██████████| 36/36 [00:00<00:00, 3063.03it/s]

Epoch 300 		 Training Loss:      0.111 		 Validation Loss:      1.108
		 Training Accuracy:    97.492% 		 Validation Accuracy:    71.842%


In [11]:
final_classifier = DNNClassifier(n_classes=12,input_size=X_train.shape[1]).to(device)
final_classifier.load_state_dict(best_model)


<All keys matched successfully>

In [12]:
_, class_report, confusion_matrix=final_classifier.evaluate_classifier(X_test, y_test)

Accuracy: 0.72


/Users/danielrodriguez/Desktop/kaggle_nlp_CS/src/classifier.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X).float()


In [13]:
print(class_report)

              precision    recall  f1-score   support

           0       0.63      0.84      0.72        97
           1       0.96      0.51      0.67        96
           2       0.55      0.92      0.69       171
           3       0.96      0.77      0.86       105
           4       0.93      0.51      0.66        82
           5       0.40      0.08      0.14        24
           6       0.87      0.73      0.80       165
           7       0.94      0.68      0.79       119
           8       0.65      0.79      0.71        76
           9       0.69      0.62      0.65        47
          10       0.91      0.52      0.66        75
          11       0.55      0.93      0.69        83

    accuracy                           0.72      1140
   macro avg       0.75      0.66      0.67      1140
weighted avg       0.78      0.72      0.72      1140



In [15]:
# Display class encoding
print(dict(zip(list(range(12)),label_encoder.classes_)))

{0: 'Education', 1: 'Entertainment', 2: 'Environment', 3: 'Fashion', 4: 'Finance', 5: 'Food', 6: 'Health', 7: 'Politics', 8: 'Science', 9: 'Sports', 10: 'Technology', 11: 'Travel'}


In [16]:
test_predictions = final_classifier.predict(X_test)

In [17]:
test_prediction_classes = label_encoder.inverse_transform(test_predictions)

In [19]:
pd.DataFrame({"ID":list(range(len(test_prediction_classes))),"Label":list(test_prediction_classes)}).to_csv("data/submission_roberta.csv", index=False)